In [9]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression

In [7]:
des_concat_train = np.load('./features/descriptions_train_concat_embed_vectors.npy')
des_concat_test = np.load('./features/descriptions_test_concat_embed_vectors.npy')
des_long_train = np.load('./features/descriptions_train_long_embed_vectors.npy')
des_long_test = np.load('./features/descriptions_test_long_embed_vectors.npy')
image_1000_train = np.load('./features/image_features_1000_train.npy')
image_1000_test = np.load('./features/image_features_1000_test.npy')
image_2048_train = np.load('./features/image_features_2048_train.npy')
image_2048_test = np.load('./features/image_features_2048_test.npy')

IOError: Failed to interpret file './features/descriptions_train_concat_embed_vectors.npy' as a pickle

In [8]:
def get_nearest_20(vec, vec_set):
    dis = [np.linalg.norm(np.array(vec) - np.array(x)) for x in vec_set]
    return np.argsort(dis)[:20]

def map20score(y, pred):
    if y in pred:
        return (20 - pred.tolist().index(y)) / 20.0
    else:
        return 0

def map20eval(pls, des_vectors, image_vectors):
    image_pred = pls.predict(des_vectors)
    top_20 = [get_nearest_20(vec, image_vectors) for vec in image_pred]
    print(len(top_20))
    scores = [map20score(i, top_20[i]) for i in range(len(top_20))]
    print "score: %f" % np.mean(scores)
    return scores

In [4]:
pls_concat_1000 = PLSRegression(n_components=256)
pls_concat_1000.fit(des_concat_train, image_1000_train)

/anaconda2/lib/python2.7/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [6]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]

In [12]:
def output_submission(top_20, output_path):
    top_20_image_IDs = map(lambda x: ' '.join(map(lambda xx: str(xx) + '.jpg', x)), top_20)
    description_ID = map(lambda x: str(x) + '.txt', range(len(top_20)))
    submission_df = pd.DataFrame({'Descritpion_ID': description_ID, 'Top_20_Image_IDs': top_20_image_IDs})
    submission_df.to_csv(output_path, index=False)


In [13]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000.csv')

In [14]:
pls_concat_1000 = PLSRegression(n_components=256, max_iter=1000)
pls_concat_1000.fit(des_concat_train, image_1000_train)

PLSRegression(copy=True, max_iter=1000, n_components=256, scale=True,
       tol=1e-06)

In [15]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]


In [16]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000_1000iter.csv')

In [40]:
des_BOW_noun_train = np.load('features/descriptions_train_BOW_noun.npy')
des_BOW_noun_test = np.load('features/descriptions_test_BOW_noun.npy')

In [56]:
pls_BOW_noun_1000 = PLSRegression(n_components=10)
pls_BOW_noun_1000.fit(des_BOW_noun_train[:8000], image_1000_train[:8000])
print pls_BOW_noun_1000.score(des_BOW_noun_train[8000:], image_1000_train[8000:])

0.3791097044896606


In [49]:
scores = map20eval(pls_BOW_noun_1000, des_BOW_noun_train[8000:], image_1000_train[8000:])

2000
score: 0.290600


In [51]:
pls_BOW_noun_1000 = PLSRegression(n_components=256)
pls_BOW_noun_1000.fit(des_BOW_noun_train, image_1000_train)

PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [52]:
pred_BOW_noun_1000 = pls_BOW_noun_1000.predict(des_BOW_noun_test)
top_20_BOW_noun_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_BOW_noun_1000]

In [53]:
output_submission(top_20_BOW_noun_1000, 'submissions/PLSR_256_BOW_noun_1000.csv')

In [59]:
des_BOW_noun_train = np.load('features/descriptions_train_BOW_noun_5452.npy')
des_BOW_noun_test = np.load('features/descriptions_test_BOW_noun_5452.npy')

In [60]:
from sklearn.decomposition import PCA

In [89]:
pca = PCA(n_components=512)

In [90]:
pca.fit(des_BOW_noun_train)

PCA(copy=True, iterated_power='auto', n_components=512, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [91]:
des_BOW_noun_train_pca = pca.transform(des_BOW_noun_train)
des_BOW_noun_test_pca = pca.transform(des_BOW_noun_test)

In [92]:
print des_BOW_noun_train_pca.shape

(10000, 512)


In [93]:
pls_BOW_noun_1000 = PLSRegression(n_components=20)
pls_BOW_noun_1000.fit(des_BOW_noun_train_pca[:8000], image_1000_train[:8000])
print pls_BOW_noun_1000.score(des_BOW_noun_train_pca[8000:], image_1000_train[8000:])

0.43198989225672163


In [96]:
pls_BOW_noun_1000 = PLSRegression(n_components=256)
pls_BOW_noun_1000.fit(des_BOW_noun_train_pca, image_1000_train)

PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [97]:
pred_BOW_noun_1000_5452 = pls_BOW_noun_1000.predict(des_BOW_noun_test_pca)
top_20_BOW_noun_1000_5452 = [get_nearest_20(vec, image_1000_test) for vec in pred_BOW_noun_1000_5452]

In [99]:
output_submission(top_20_BOW_noun_1000_5452452, 'submissions/PLSR_256_BOW_noun_1000_5452_pca_512.csv')

In [100]:
"""
Luffy part.
"""

import pickle
import os.path
from sklearn.decomposition import PCA

train_description_feature_map = pickle.load(open('./features/11_30_[train]_description_normalized_only_>1_vector_py27.pkl', 'rb'))
test_derscription_feature_map = pickle.load(open('./features/11_30_[test]_description_normalized_only_>1_vector_py27.pkl', 'rb'))

train_bow_lst = list(map(lambda x: x["BOW_all_normalized_vector"], train_description_feature_map))
test_bow_lst = list(map(lambda x: x["BOW_all_normalized_vector"], test_derscription_feature_map))

pca_model_name = "./models/pca_bow_all_to_2048.pkl"
if os.path.exists(pca_model_name):
    pca = pickle.load(open(pca_model_name, 'rb'))
    print "loaded pca model..."
else:
    # train PCA.
    pca = PCA(n_components=3500)
    pca.fit(train_bow_lst)
    pickle.dump(pca, open(pca_model_name, 'wb'), protocol=2)
    print "trained and saved pca model..."

des_BOW_all_train_pca = pca.transform(train_bow_lst)
des_BOW_all_test_pca = pca.transform(test_bow_lst)

print des_BOW_all_test_pca.shape

(2000, 2048)


In [81]:
train_img_map = pickle.load(open('./features/12_1_train_img_feature_map_py27.pkl', 'rb'))
test_img_map = pickle.load(open('./features/12_1_test_img_feature_map_py27.pkl', 'rb'))
print "loaded img feature map..."

In [82]:
train_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], train_img_map)))
test_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], test_img_map)))

In [99]:
model_name = "./models/pls_800_pool5_to_BOW_all_pca_2048.pkl"
if os.path.exists(model_name):
    model = pickle.load(open(model_name, 'rb'))
else:
    # train PLSR.
    model = PLSRegression(n_components=800)
    model.fit(train_pool5_img[:8000], des_BOW_all_train_pca[:8000])
    pickle.dump(model, open(model_name, 'wb'), protocol=2)
    print model.score(train_pool5_img[8000:], des_BOW_all_train_pca[8000:])
print "model loaded..."

model loaded...


In [101]:
from scipy.spatial.distance import cdist
import csv

pred_test_pool5 = model.predict(test_pool5_img)
dist = cdist(des_BOW_all_test_pca, pred_test_pool5, 'cosine') # CHANGE!!


print("description * images dist matrix, shape:", dist.shape)
sorted_id = np.argsort(dist) # dist: N_description * N_images dist matrix.

with open('./submission/submission_plsr_800_pool5_to_bow_all_pca_2048.csv', 'w') as csvfile:
        # write csv header
        fieldnames = ['Descritpion_ID', "Top_20_Image_IDs"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for i, row in enumerate(sorted_id):
            top_choices =  list(map(lambda x: str(x) + ".jpg", row[:20]))
            res = {}
            res['Descritpion_ID'] = str(i) + ".txt" # file name
            res['Top_20_Image_IDs'] = " ".join(top_choices)
            writer.writerow(res)

print("Writing Complete.")

('description * images dist matrix, shape:', (2000, 2000))
Writing Complete.


In [112]:
"""
Text preprocessing.
"""
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    return words


[nltk_data] Downloading package stopwords to /Users/EL-
[nltk_data]     CAPITAN-2016/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/EL-
[nltk_data]     CAPITAN-2016/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [163]:
"""
Compute TFIDF for test_derscription_feature_map "u'description_complete"
"""
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
documents = list(map(lambda x: x["description_all_cleaned"], test_derscription_feature_map))
dictionary = Dictionary(documents)

tf_model_name = "./models/TFIDF_test_description_all_cleaned.tf"
if os.path.exists(tf_model_name):
    tfidf_model = TfidfModel.load(tf_model_name)
    print "TFIDF model loaded..."
else:
    tfidf_model = TfidfModel([dictionary.doc2bow(d) for d in documents], id2word=dictionary)
    tfidf_model.save("./models/TFIDF_test_description_all_cleaned.tf")
    print "TFIDF model saved..."
    
tfidf_values = dict(tfidf_model[dictionary.doc2bow(test_derscription_feature_map[0]["description_all_cleaned"])])
 
print tfidf_values[dictionary.token2id['woman']]                     # 0.0367516096888
print tfidf_values[dictionary.token2id['walk']]                # 0.0538505795815
print tfidf_values[dictionary.token2id['street']]                 # 0.0683210467787
print tfidf_values[dictionary.token2id['watch']]               # 0.0945807226371
print tfidf_values[dictionary.token2id['phone']]                 # 0.107960637598

TFIDF model loaded...
0.12499730496979501
0.14684655407981254
0.1257889789279195
0.19298520329431507
0.20056456953600987


In [122]:
"""
Use tags overlap to re-rank.

- test_derscription_feature_map ['description_n_cleaned']
- count image tags total apperances / ['description_n_cleaned'] length in above set. 
- add to dist matrix, then sorting.
"""
# print test_derscription_feature_map[0]
for t in test_derscription_feature_map:
    words = t["description_complete"].split(" ")
    words = normalize(words)
    t["description_all_cleaned"] = words

print test_derscription_feature_map[0]

{u'description_complete': u'A woman walking down a street past a doorway. a woman is walking past a doorway on a sidewalk A woman talking on a cell phone checks her watch. A woman talks on a cell phone and checks her watch while walking on a sidewalk. A woman wearing blue is on the phone and walking along a sidewalk in front of a building with black planters on both sides of the entrance.', 'description_all_cleaned': [u'woman', u'walk', u'street', u'past', u'doorway', u'woman', u'walk', u'past', u'doorway', u'sidewalk', u'woman', u'talk', u'cell', u'phone', u'check', u'watch', u'woman', u'talk', u'cell', u'phone', u'check', u'watch', u'walk', u'sidewalk', u'woman', u'wear', u'blue', u'phone', u'walk', u'along', u'sidewalk', u'front', u'build', u'black', u'planters', u'side', u'entrance'], u'name': u'0.txt', u'description_n_v_adj': u'woman street doorway. woman doorway sidewalk woman cell phone watch. woman cell phone watch sidewalk. woman blue phone walking sidewalk front building blac

In [164]:
from tqdm import tqdm

test_img_tags = pickle.load(open("./features/12_1_test_img_tags.pkl", "rb"))

new_dist = np.zeros(dist.shape)

def description_map_to_tags(words):
    """
    A set of rules to map certain descriptions nouns into tags wordding.
    such as:
    "man", "guy", "woman" => "person"
    """
    new_words = set()
#     for w in words:
#         if w == "man" or w == "woman" or w == "guy":
#             new_words.add("person")
#         else:
#             new_words.add(w)
    for w in words:
        new_words.add(w)
    return list(new_words)

test_img_caption_cleaned_set = []
for img_id, obj in enumerate(test_img_captions):
    obj["caption_cleaned"] = normalize(test_img_captions[img_id]["caption"].split(" "))
    test_img_caption_cleaned_set.append(obj)

for d_id, description_row in tqdm(enumerate(dist)):
    description_cleaned = test_derscription_feature_map[d_id]["description_all_cleaned"]
    description_cleaned = description_map_to_tags(description_cleaned) # hand-crafted map book.
    test_derscription_feature_map[d_id]["description_all_cleaned"] = description_cleaned
    sorted_ids = np.argsort(description_row)
    for i_id, img_id in enumerate(sorted_ids):
        img_tags = test_img_tags[img_id]["tags"]
        caption_cleaned = test_img_caption_cleaned_set[img_id]["caption_cleaned"]
        tag_count_in_description = 0
        for tag in caption_cleaned:
            tag_count_in_description += description_cleaned.count(tag)
        this_img_score = float(tag_count_in_description) / len(description_cleaned)
        new_dist[d_id][i_id] = dist[d_id][i_id] -  this_img_score # re-rank dist matrix by adding tag overlap information.


0it [00:00, ?it/s]
11it [00:00, 104.72it/s]
22it [00:00, 105.24it/s]
30it [00:00, 95.32it/s] 
39it [00:00, 91.48it/s]
49it [00:00, 92.53it/s]
59it [00:00, 94.06it/s]
69it [00:00, 95.17it/s]
80it [00:00, 98.47it/s]
91it [00:00, 100.27it/s]
101it [00:01, 99.01it/s]
111it [00:01, 97.21it/s]
122it [00:01, 98.57it/s]
132it [00:01, 98.65it/s]
142it [00:01, 98.63it/s]
152it [00:01, 97.96it/s]
162it [00:01, 98.52it/s]
173it [00:01, 99.85it/s]
184it [00:01, 100.28it/s]
195it [00:01, 100.67it/s]
206it [00:02, 101.68it/s]
217it [00:02, 101.95it/s]
228it [00:02, 101.78it/s]
239it [00:02, 101.26it/s]
250it [00:02, 102.31it/s]
261it [00:02, 102.36it/s]
272it [00:02, 102.97it/s]
283it [00:02, 103.65it/s]
294it [00:02, 103.73it/s]
305it [00:03, 102.45it/s]
316it [00:03, 102.37it/s]
327it [00:03, 98.97it/s] 
337it [00:03, 97.16it/s]
347it [00:03, 93.98it/s]
357it [00:03, 95.41it/s]
367it [00:03, 95.91it/s]
377it [00:03, 94.10it/s]
388it [00:03, 96.70it/s]
398it [00:04, 96.22it/s]
408it [00:04, 94.69it

In [165]:
test_id = 9

print dist[test_id]
print np.argsort(dist)[test_id][:10]
print("after re-ranking...")
print new_dist[test_id]
print np.argsort(new_dist)[test_id][:10]

[1.11061263 0.94336359 0.9853039  ... 1.08531465 0.93662826 1.05280569]
[ 570  549  899  712 1886 1371 1700  770 1795  614]
after re-ranking...
[0.74748233 0.58023329 0.62217361 ... 1.08531465 0.93662826 1.05280569]
[549 101   5 624 222  95 689 412 146 408]


In [90]:
test_img_captions = pickle.load(open("./features/12_1_test_image_caption.pkl", "rb"))

In [158]:
"""
This section experiments on fine tune on tags.
"""
img_id = 689
description_id = 7

print "test img tags:", test_img_tags[img_id]["tags"]
print "test img origin caption:", test_img_captions[img_id]
test_img_caption_cleaned_set = normalize(test_img_captions[img_id]["caption"].split(" "))
print "test img caption - cleaned", test_img_caption_cleaned_set
print "\n"
# print "test img description:", test_derscription_feature_map[description_id]["description_all_cleaned"]

test_img_description_cleaned_set = description_map_to_tags(test_derscription_feature_map[description_id]["description_all_cleaned"])
print "test img description - cleaned:", test_img_description_cleaned_set
print "\n"

tfidf_values = dict(tfidf_model[dictionary.doc2bow(test_derscription_feature_map[description_id]["description_all_cleaned"])])

# score for common tags:
this_img_apperance_in_description = 0
for tag in test_img_tags[img_id]["tags"]:
    this_img_apperance_in_description += test_img_description_cleaned_set.count(tag)
this_img_score = float(this_img_apperance_in_description) / len(test_img_description_cleaned_set)
print "overlap score for simple tag:", this_img_score

# score for caption tags:
this_img_apperance_in_description = 0
for tag in test_img_caption_cleaned_set:
    if tag in test_img_description_cleaned_set:
        w = tfidf_values[dictionary.token2id[tag]] 
        this_img_apperance_in_description += w
#     this_img_apperance_in_description += test_img_description_cleaned_set.count(tag)
this_img_score = float(this_img_apperance_in_description) / len(test_img_caption_cleaned_set)
print "overlap score for caption tag:", this_img_score

test img tags: ['zebra']
test img origin caption: {'caption_cleaned': [u'group', u'zebra', u'stand', u'top', u'lush', u'green', u'field'], u'caption': u'a group of zebra standing on top of a lush green field .', u'name': u'689.jpg'}
test img caption - cleaned [u'group', u'zebra', u'stand', u'top', u'lush', u'green', u'field']


test img description - cleaned: [u'exhibit', u'lawn', u'toward', u'grassy', u'show', u'pose', u'wildlife', u'young', u'next', u'stand', u'man', u'build', u'guy', u'grass', u'gesture', u'zebra']


overlap score for simple tag: 0.0625
overlap score for caption tag: 0.333449253322


In [76]:
new_sorted_id = np.argsort(new_dist)
with open('./submission/submission_plsr_600_pool5_to_bow_all_pca_1024+rerank_by_tag_overlap.csv', 'w') as csvfile:
    # write csv header
    fieldnames = ['Descritpion_ID', "Top_20_Image_IDs"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for i, row in enumerate(new_sorted_id):
        top_choices =  list(map(lambda x: str(x) + ".jpg", row[:20]))
        res = {}
        res['Descritpion_ID'] = str(i) + ".txt" # file name
        res['Top_20_Image_IDs'] = " ".join(top_choices)
        writer.writerow(res)

print("Writing Complete.")

Writing Complete.


In [48]:
# import os
# test_img_tags = []
# test_image_tag_base = "./../../all/data/tags_test/"
# for file in os.listdir(test_image_tag_base):
#     if file.endswith(".txt"):
#         obj = {}
#         obj["name"] = file
#         index = file.split(".")[0]
#         obj["id"] = int(index)
#         file_name = test_image_tag_base + file
#         with open(file_name) as f:
#             content = f.readlines()
#             content = [x.strip().split(":")[1].split(" ") for x in content]
#             content = [item for sublist in content for item in sublist]
#         obj["tags"] = content
#         test_img_tags.append(obj)
# test_img_tags = sorted(test_img_tags, key=lambda x:x["id"])
# pickle.dump(test_img_tags, open("./features/12_1_test_img_tags.pkl", 'wb'), protocol=2)
# print test_img_tags[0]

{'tags': ['bed', 'backpack', 'suitcase', 'tie'], 'name': '0.txt', 'id': 0}


In [89]:

# test_img_captions = sorted(test_img_captions, key=lambda x: int(x["name"].split(".")[0]))
# pickle.dump(test_img_captions, open("./features/12_1_test_image_caption.pkl", 'wb'), protocol=2)